In [1]:
with open('CV.tex','r') as fp:
    lines=fp.readlines()

In [2]:
import re,os

In [3]:
CV_items=dict()
for i,l in enumerate(lines):
    if l.startswith('\itemtitle'):
        item = re.split('{|}',l)[1]
        istr = i
        CV_items[item]=dict(istr=istr)
    if l.startswith('\end{document}'):
        iend = istr
        CV_items[item]['iend']=i

itemlist=list(CV_items.keys())

for item1, item2 in zip(itemlist[:-1],itemlist[1:]):
    CV_items[item1].update(dict(iend=CV_items[item2]['istr']))

In [4]:
for item in itemlist:
    
    c=CV_items[item]
    content=[]
    element=[]
    for l in lines[c['istr']:c['iend']]:
        if ('itemtitle' in l) | (l == '\n') | l.startswith('%'): continue
        if ('element' in l) & (len(element)>0): 
            content.append(''.join(element).replace('\n',''))
            element=[]
        element.append(l)
    content.append(''.join(element).replace('\n',''))    
    
    c.update(dict(content=content))

In [5]:
l=c['content'][1]
def get_content_split(l,level=1):
    lbrackets=dict()
    rbrackets=dict()
    lev=0
    for i,w in enumerate(l):
        if w == '{':
            left = i
            lev += 1
            if lev in lbrackets:
                lbrackets[lev].append(left)
            else:
                lbrackets[lev] = [left]
        if w == '}':
            right = i
            if lev in rbrackets:
                rbrackets[lev].append(right)
            else:
                rbrackets[lev] = [right]
            lev -= 1
    sp = [l[i+1:j].strip() for i,j in zip(lbrackets[level],rbrackets[level])]
    return sp

In [6]:
import json

In [7]:
# Education/Current Position/Employment
for item in itemlist[:10]:
    print(item)
    c = CV_items[item]
    items = []
    for l in c['content']:
        sp = get_content_split(l,level=1)
#         print(sp)
        years = sp[0].replace('\\\\','').split('--')
        y1 = years[0].strip()
        try:
            y2 = years[1].strip()
            if (len(y2) == 0): y2='present'
        except IndexError:
            y2 = 'None'

        title = sp[1].split(',')[0]
        other = sp[-1]
        items.append(dict(titie=title,y1=y1,y2=y2,other=other))
    json_name = item.lower().replace('/',' and ').replace(' ','_').replace('-','')+'.json'
    with open(os.path.join('../data',json_name),'w') as fp:
        json.dump(items,fp,indent=4, separators=(",", ": "))

Education
Current Position
Employment
Grants
Research Advising
Teaching
Observing Proposals
Computing Time Allocations
Scientific Collaboration Teams
Professional Service


In [9]:
for item in itemlist[-3:]:
    c = CV_items[item]
    items = []
    for l in c['content']:
        sp = get_content_split(l,level=1)
    #         print(sp)
        years = sp[0].replace('\\\\','').split('--')
        y1 = years[0].strip()
        try:
            y2 = years[1].strip()
            if (len(y2) == 0): y2='present'
        except IndexError:
            y2 = 'None'

        talk = sp[1].split(',')[0]
        other = sp[-1].split(',')
        title, = get_content_split(','.join(other[:-3]))
        conf = other[-3].strip()
        if 'href' in conf:
            conf = get_content_split(conf)
            conf_url = conf[0].strip()
            conf = conf[1].strip()
        venue = ','.join(other[-2:]).strip()

        items.append(dict(talk=talk,y1=y1,y2=y2,title=title,conf=conf,venue=venue))

    json_name = '_'.join(item.split(' ')[1:]).lower().replace('/','_')+'.json'
    print(json_name)
    with open(os.path.join('../data',json_name),'w') as fp:
        json.dump(items,fp,indent=4, separators=(",", ": "))

invited_review_talks.json
invited_colloquia.json
conference_workshop_seminar_talks.json
